In [8]:
from pathlib import Path
from datetime import datetime
import json, hashlib, io, re
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# project root = the folder that contains this notebook
ROOT = Path.cwd()
RAW = ROOT / "data" / "raw"
RAW.mkdir(parents=True, exist_ok=True)

START = "2015-01-01"
END   = "2024-12-31"

In [9]:
# Helper dirs, hashing, metadata
def sha256(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

def write_metadata(csv_path: Path, meta: dict):
    meta = dict(meta)  # copy
    meta["file_size_bytes"] = csv_path.stat().st_size
    meta["sha256"] = sha256(csv_path)
    meta_path = csv_path.with_name(csv_path.stem + "_metadata.json")
    meta_path.write_text(json.dumps(meta, indent=2), encoding="utf-8")
    return meta_path

def session_with_retries() -> requests.Session:
    s = requests.Session()
    retries = Retry(
        total=5,
        backoff_factor=0.6,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"]),
        raise_on_status=False,
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.headers.update({"User-Agent": "interim-report/1.0 (requests)"})
    return s


In [10]:
KEY_FILE = ROOT / "fred_api_key.txt"
raw = KEY_FILE.read_text(encoding="utf-8-sig").strip()
# strip straight/smart quotes + any whitespace just in case
api_key = re.sub(r"\s+", "", raw.strip().strip('"').strip("'").strip("“”‘’"))
assert api_key and api_key.isalnum(), "Your API key looks malformed. Type it by hand, no quotes/spaces."
len(api_key), api_key[:4] + "…"  # quick peek


(32, 'bc3f…')

In [11]:
FRED_OBS_URL = "https://api.stlouisfed.org/fred/series/observations"

params = {
    "series_id": "CPIAUCSL",
    "api_key": api_key,
    "file_type": "json",
    "observation_start": START,
    "observation_end": END,
    "sort_order": "asc",
}

sess = session_with_retries()
r = sess.get(FRED_OBS_URL, params=params, timeout=60)
r.raise_for_status()
obs = r.json().get("observations", [])
cpi = pd.DataFrame(obs)[["date", "value"]]
cpi["value"] = pd.to_numeric(cpi["value"], errors="coerce")
cpi.rename(columns={"value": "cpi"}, inplace=True)

cpi_path = RAW / "CPIAUCSL.csv"
cpi.to_csv(cpi_path, index=False)
meta_cpi = write_metadata(
    cpi_path,
    {
        "series_id": "CPIAUCSL",
        "source": "FRED API",
        "endpoint": FRED_OBS_URL,
        "observation_start": START,
        "observation_end": END,
        "retrieved_at_utc": datetime.utcnow().isoformat(timespec="seconds")+"Z",
        "row_count": int(cpi.shape[0]),
        "columns": list(cpi.columns),
    },
)
cpi.head(), cpi.shape, cpi_path, meta_cpi


/var/folders/k3/17j6sc1951d97g65z3fv2_rm0000gn/T/ipykernel_42199/746266452.py:30: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "retrieved_at_utc": datetime.utcnow().isoformat(timespec="seconds")+"Z",


(         date      cpi
 0  2015-01-01  234.747
 1  2015-02-01  235.342
 2  2015-03-01  235.976
 3  2015-04-01  236.222
 4  2015-05-01  237.001,
 (120, 2),
 PosixPath('/Users/shihaoyu/Desktop/interim_report/data/raw/CPIAUCSL.csv'),
 PosixPath('/Users/shihaoyu/Desktop/interim_report/data/raw/CPIAUCSL_metadata.json'))

In [14]:
import io, sys

def get_pce_csv() -> pd.DataFrame:
    sess = session_with_retries()

    # 1) Primary: official “downloaddata” CSV (no API key needed)
    urls = [
        ("primary", "https://fred.stlouisfed.org/series/PCE/downloaddata/PCE.csv", None),
        # 2) Fallback: fredgraph CSV with id parameter
        ("fallback", "https://fred.stlouisfed.org/graph/fredgraph.csv", {"id": "PCE"}),
    ]

    last_err = None
    for label, url, params in urls:
        try:
            r = sess.get(
                url, params=params, timeout=60,
                headers={"Accept": "text/csv, */*"}
            )
            # helpful debug if something is odd
            print(f"[{label}] status={r.status_code} url={r.url} content-type={r.headers.get('content-type')}")
            r.raise_for_status()

            # Sometimes CDNs return HTML; check and show a snippet if so
            head_txt = r.content[:512].decode("utf-8", errors="ignore").lower()
            if "<html" in head_txt or "DOCTYPE html".lower() in head_txt:
                raise ValueError(f"{label}: server returned HTML instead of CSV. First bytes: {head_txt[:120]}...")

            # Try to read as CSV
            df = pd.read_csv(io.BytesIO(r.content))
            # Accept common headers like DATE, VALUE or observation_date, PCE
            normalized_cols = [c.strip().lower() for c in df.columns]
            if not (("date" in normalized_cols or "observation_date" in normalized_cols) and
                    ("value" in normalized_cols or "pce" in normalized_cols)):
                raise ValueError(f"{label}: unexpected columns {df.columns.tolist()}")

            # Standardize to date/value
            colmap = {}
            if "observation_date" in normalized_cols:
                colmap[df.columns[normalized_cols.index("observation_date")]] = "date"
            if "date" in normalized_cols:
                colmap[df.columns[normalized_cols.index("date")]] = "date"
            if "value" in normalized_cols:
                colmap[df.columns[normalized_cols.index("value")]] = "value"
            if "pce" in normalized_cols:
                colmap[df.columns[normalized_cols.index("pce")]] = "value"

            df = df.rename(columns=colmap)[["date", "value"]]
            return df

        except Exception as e:
            print(f"[warn] {label} failed: {e}")
            last_err = e

    raise RuntimeError(f"PCE download failed from all endpoints: {last_err}")

# ---- run and save ----
pce = get_pce_csv()
pce.columns = ["date", "pce"]
pce_path = RAW / "PCE.csv"
pce.to_csv(pce_path, index=False)
meta_pce = write_metadata(
    pce_path,
    {
        "source": "FRED download (CSV)",
        "endpoints_tried": [
            "https://fred.stlouisfed.org/series/PCE/downloaddata/PCE.csv",
            "https://fred.stlouisfed.org/graph/fredgraph.csv?id=PCE",
        ],
        "retrieved_at_utc": datetime.utcnow().isoformat(timespec="seconds")+"Z",
        "row_count": int(pce.shape[0]),
        "columns": list(pce.columns),
    },
)
pce.head(), pce.shape, pce_path, meta_pce


[warn] primary failed: HTTPSConnectionPool(host='https', port=443): Max retries exceeded with url: /fred.stlouisfed.org/docs/api/fred/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x12f7eec90>: Failed to resolve 'https' ([Errno 8] nodename nor servname provided, or not known)"))
[fallback] status=200 url=https://fred.stlouisfed.org/graph/fredgraph.csv?id=PCE content-type=application/csv


/var/folders/k3/17j6sc1951d97g65z3fv2_rm0000gn/T/ipykernel_42199/1749431965.py:70: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "retrieved_at_utc": datetime.utcnow().isoformat(timespec="seconds")+"Z",


(         date    pce
 0  1959-01-01  306.1
 1  1959-02-01  309.6
 2  1959-03-01  312.7
 3  1959-04-01  312.2
 4  1959-05-01  316.1,
 (800, 2),
 PosixPath('/Users/shihaoyu/Desktop/interim_report/data/raw/PCE.csv'),
 PosixPath('/Users/shihaoyu/Desktop/interim_report/data/raw/PCE_metadata.json'))

In [15]:
# Convert to datetime and inner join on date
cpi["date"] = pd.to_datetime(cpi["date"])
pce["date"] = pd.to_datetime(pce["date"])

merged = (
    pd.merge(cpi, pce, on="date", how="inner")
    .sort_values("date")
    .reset_index(drop=True)
)
merged.tail(), merged.shape


(          date      cpi      pce
 115 2024-08-01  314.131  20001.3
 116 2024-09-01  314.851  20147.6
 117 2024-10-01  315.564  20226.0
 118 2024-11-01  316.449  20313.6
 119 2024-12-01  317.603  20514.3,
 (120, 3))